In [5]:
import pandas as pd
from pandas import Series, DataFrame

#removes all paranthesis and commas from the data, convert to float values
#assumes that a an opening paranthesis indicates a negative number
def make_float(x):
    if type(x) is float: #ignore exsiting 'nan'/float values
        return
    if "-" in x: #dash means there is no data for that year
        return float('nan')
    return float(x.replace("(", "-").replace(",","").replace(")",""))

city_budget = pd.read_csv('city_budget_data.csv')

city_budget["2015"] = city_budget["2015"].apply(make_float)
city_budget["2016"] = city_budget["2016"].apply(make_float)
city_budget["2017"] = city_budget["2017"].apply(make_float)
city_budget["2018"] = city_budget["2018"].apply(make_float)

fire_budget = city_budget[city_budget["Agency"] == "30 -  FIRE"]
fire_budget.reset_index(drop=True)

,Agency,Organization,Account Type,Character Code,Object,2015,2016,2017,2018
0,30 - FIRE,30252 - FIRE INSPECTION ELEVATOR,Revenue,44 - LICENSES AND PERMITS,44230 - ELEVATOR PERMITS AND INSPECTS,-617670.0,-606447.0,-534775.0,-555937.0
1,30 - FIRE,30240 - FIRE GENERAL PREVENTION,Revenue,44 - LICENSES AND PERMITS,44245 - FIRE PERMITS,-396640.0,-416396.0,-401755.0,-553480.0
2,30 - FIRE,30240 - FIRE GENERAL PREVENTION,Revenue,47 - MISC REVENUE,47190 - MISCELLANEOUS REVENUE,-95363.0,-103890.0,-92455.0,-135419.0
3,30 - FIRE,30120 - FIRE MEDICAL PROGRAM GENERAL,Revenue,43 - CHARGES FOR SERVICES,43225 - SPECIAL DUTY,-83384.0,-104093.0,-128038.0,-73087.0
4,30 - FIRE,30114 - FIRE HAZARDOUS INCIDENT TEAM,Revenue,42 - INTERGOV REVENUES,42410 - OTHER UNIT OF GOV REVENUES OP,-57023.0,-61862.0,-56914.0,-45007.0
5,30 - FIRE,30110 - FIRE GEN FIELD OP,Revenue,42 - INTERGOV REVENUES,42210 - STATE REVENUES OPERATING,-47469.0,NaN,NaN,NaN
6,30 - FIRE,30110 - FIRE GEN FIELD OP,Revenue,43 - CHARGES FOR SERVICES,43710 - REIMBURSEMENT OF EXPENSE,-44871.0,-58561.0,-41825.0,-57251.0
7,30 - FIRE,30120 - FIRE MEDICAL PROGRAM GENERAL,Revenue,43 - CHARGES FOR SERVICES,43710 - REIMBURSEMENT OF EXPENSE,-35083.0,-68402.0,-65906.0,-48118.0
8,30 - FIRE,30121 - FIRE MEDICAL PROGRAM STATE,Revenue,42 - INTERGOV REVENUES,42210 - STATE REVENUES OPERATING,-24649.0,-28347.0,-25266.0,-22656.0
9,30 - FIRE,30251 - FIRE INSPECTION TANK PROGRAM,Revenue,42 - INTERGOV REVENUES,42210 - STATE REVENUES OPERATING,-24610.0,-24887.0,-28602.0,-29697.0
